In [1]:
%pip install transformers datasets peft
%pip install accelerate -U


DEPRECATION: xgbse 0.2.3 has a non-standard dependency specifier pandas>=1.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of xgbse or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: xgbse 0.2.3 has a non-standard dependency specifier pandas>=1.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of xgbse or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer and model
model_name = "distilgpt2"
model_path = None #'./finetuned_distilgpt2_subset'
save_dir = "./finetuned_distilgpt2"

if model_path is not None:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)   
tokenizer.padding_side = "left"  # Set padding to the left side

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id


/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import DataCollatorForLanguageModeling

loaded_dataset_dict = load_from_disk('./dataset_dict')

# Create a dataset
train_dataset = loaded_dataset_dict['train']

# Use Max allowed context length from GPT2
max_length = 1024

# Step 3: Tokenize the dataset using the dynamic max_length
def tokenize_function(examples):
    encodings = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=max_length)
    encodings['labels'] = encodings['input_ids']
    return encodings

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

In [4]:
subset_size = None  # Number of examples you want to select

if subset_size is not None:
    # Shuffle the dataset
    tokenized_dataset = tokenized_dataset.shuffle(seed=42)

    # Select a random subset of the dataset
    tokenized_dataset = tokenized_dataset.select(range(subset_size))

In [5]:
from tqdm import tqdm
import numpy as np
# Function to produce summary statistics of tokenized lengths for texts containing "cillabel"
def print_examples_with_tokenized_lengths(dataset, tokenizer, filter_keyword="cillabel"):
    # Filter dataset for entries containing the keyword
    filtered_examples = dataset.filter(lambda example: filter_keyword in example['text'].lower())
    
    # Extract tokenized lengths
    tokenized_lengths = []
    for example in tqdm(filtered_examples):
        text = example['text']
        tokenized_output = tokenizer(text, truncation=False)
        tokenized_length = len(tokenized_output['input_ids'])
        tokenized_lengths.append(tokenized_length)
    
    if tokenized_lengths:
        # Calculate summary statistics
        length_stats = {
            "count": len(tokenized_lengths),
            "mean": np.mean(tokenized_lengths),
            "std": np.std(tokenized_lengths),
            "min": np.min(tokenized_lengths),
            "25th percentile": np.percentile(tokenized_lengths, 25),
            "median": np.median(tokenized_lengths),
            "75th percentile": np.percentile(tokenized_lengths, 75),
            "max": np.max(tokenized_lengths)
        }
        
        # Print statistics
        print(f"Summary statistics for texts containing '{filter_keyword}':")
        for stat, value in length_stats.items():
            print(f"{stat.capitalize()}: {value}")
    else:
        print(f"No examples found containing '{filter_keyword}'.")

# Generate summary statistics for texts containing the keyword "cillabel"
print_examples_with_tokenized_lengths(train_dataset, tokenizer, filter_keyword="cillabel")

100%|██████████| 54991/54991 [00:54<00:00, 1009.74it/s]

Summary statistics for texts containing 'cillabel':
Count: 54991
Mean: 754.1586259569748
Std: 575.9800417057043
Min: 249
25th percentile: 315.0
Median: 597.0
75th percentile: 987.0
Max: 11568


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    logging_dir="./logs",
    logging_steps=500,  # Log every 10 steps
    save_steps=10000,    # Save checkpoint every 10000 steps
)


/opt/conda/envs/llm/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

# Train the model
trainer.train(resume_from_checkpoint=True)


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss
96000,1.111700
96500,1.168200
97000,1.223200
97500,1.132600
98000,1.138900
98500,1.175900
99000,1.176500
99500,1.223400
100000,1.254500
100500,1.242400


TrainOutput(global_step=287130, training_loss=0.8682099481811885, metrics={'train_runtime': 97993.518, 'train_samples_per_second': 23.439, 'train_steps_per_second': 2.93, 'total_flos': 6.001620545647411e+17, 'train_loss': 0.8682099481811885, 'epoch': 30.0})

## Conditional Generation

In [16]:
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./finetuned_distilgpt2/tokenizer_config.json',
 './finetuned_distilgpt2/special_tokens_map.json',
 './finetuned_distilgpt2/vocab.json',
 './finetuned_distilgpt2/merges.txt',
 './finetuned_distilgpt2/added_tokens.json',
 './finetuned_distilgpt2/tokenizer.json')

In [17]:
test_dataset = loaded_dataset_dict['test']


def truncate_text(text, max_length=512):
    return text[:max_length]

# Apply truncation
truncated_texts = [truncate_text(example['text']) for example in test_dataset]

In [18]:
def create_prompts(text, max_prompt_ratio=0.5):
    prompt_length = int(len(text) * max_prompt_ratio)
    return text[:prompt_length]

# Create prompts
prompts = [create_prompts(text) for text in truncated_texts]


In [19]:
prompts[0]

'on 2022-06-10 at time morning,user_id is 286442, age is 3, gender is 2, residence is 20, city is 328, city_rank is 5, series_dev is 31, series_group is 3, emui_dev is 20, device_name is 122, device_size is 2117, and 1269, net_type is 7, task_id is 10463, a'

In [20]:
def generate_completions_in_batches(prompts, model, tokenizer, max_length=256, batch_size=8):
    # Get the device of the model
    device = model.device
    completions = []
    
    # Process prompts in batches
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        
        # Tokenize and move the input to the same device as the model
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)
        
        # Generate outputs for the batch
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1
        )
        
        # Decode and collect the completions for each prompt in the batch
        batch_completions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        completions.extend(batch_completions)
    
    return completions


In [21]:
completions = generate_completions_in_batches(prompts[:4], model, tokenizer, max_length=1024, batch_size=4)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
completions

['on 2022-06-10 at time morning,user_id is 286442, age is 3, gender is 2, residence is 20, city is 328, city_rank is 5, series_dev is 31, series_group is 3, emui_dev is 20, device_name is 122, device_size is 2117, and 1269, net_type is 7, task_id is 10463, a, and 16, and 4, and 25, and 24, and 25, and 14, and 4, and 16, and 16, and and 25, and 17, and 25, and 18,,, and, and 17, and and, and 17, and,, and and,, and,, and, and 17, and, and 17,,id,, and, and 17,, and, and, and, and, and and is and,, and and,, and 14, and,, and, and,,, and 14,, and,_, and 20, and, and 17, and, and and and,, and,, and,, and,, and and, and 17, and andid isid,, and 17, and and 14,, and 14, and,, and 14, and__type, and_,,, and, and, and, and 14, and,, and and, and 16,_, and, and,, and, and 14,,, and,,, and, is and, and, and and and_,_id is, and, and, and, and, and and 14,, and and and, and and and, and and, and, andapp__id_,, and and, and is and and, is_, and, and_app, andclass_v,, and 17,list_class is is 12, 

In [15]:
# Optionally, save to a file
with open("synth_data/completions.txt", "w") as f:
    for i, completion in enumerate(completions):
        #f.write(f"Prompt {i+1}:\n{prompts[i]}\n")
        f.write(f"{completion}\n")
        #f.write("="*80 + "\n")
